# Some *tests* of the inner functionality of the whole dqn package

## (a) Testing unity ml-agents env wrapper

In [ ]:
import os
import sys
import numpy as np

from navigation.envs import mlagents

In [ ]:
# definitions for full paths to executables
EXEC_BANANA_PATH = os.path.join( os.getcwd(), 'executables/Banana_Linux/Banana.x86_64' )
EXEC_BANANA_NOVIS_PATH = os.path.join( os.getcwd(), 'executables/Banana_Linux_NoVis/Banana.x86_64' )

In [ ]:
# initialize the environment
_env = mlagents.createDiscreteActionsEnv( EXEC_BANANA_PATH )
## _env = mlagents.createDiscreteActionsEnv( EXEC_BANANA_NOVIS_PATH )

In [ ]:
print( 'numActions: ', _env.numActions )
print( 'obsShape: ', _env.obsShape )

In [ ]:
_state = _env.reset( training = False )
print( '_state.shape: ', _state.shape )
print( '_state:' )
_raysState = _state[:-2].reshape( 7, -1 )
print( '_raysState: ' )
print( _raysState )

def process( state ) :
    return state[:-2].reshape( 7, -1 )

In [ ]:
_raysState[:,[0,2,4]]

In [ ]:
_state = _env.reset( training = False )

while True :
    
    _action = np.random.randint( _env.numActions )
    _snext, _reward, _done, _ = _env.step( 2 )
    
    print( '#######################' )
    print( '_snext: ' )
    print( process( _snext ) )
    print( '_reward: ', _reward )
    print( '_done: ', _done )
    
    if _done :
        break

_env.close()

## Testing the sumtree data structure

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from navigation.dqn.utils import sumtree

In [ ]:
smtree = sumtree.SumTree(8)

smtree.add( 'a', 3 )
smtree.add( 'b', 10 )
smtree.add( 'c', 12 )
smtree.add( 'd', 4 )
smtree.add( 'e', 1 )
smtree.add( 'f', 2 )
smtree.add( 'g', 8 )
smtree.add( 'h', 2 )

In [ ]:
print( 'arr-tree: ', smtree._tree )
print( 'data: ', smtree._data )

### testing sampling method

In [ ]:
_val = 42. * np.random.random()
_indx, _nodeval, _data = smtree.getNode( _val )
print( 'sampled (%s), with nodeval=%.2f and indx=%i, for value=%.2f' % ( _data, _nodeval, _indx, _val )  )

### checking the probability distribution given by the node values

In [ ]:
NSAMPLES = 10000
BUCKETS = [0] * len( smtree._data )
for _ in range( NSAMPLES ) :
    _indx, _, _data = smtree.getNode( 42. * np.random.random() )
    BUCKETS[_indx - len( smtree._data ) + 1] += 1
    
plt.bar( ['a','b','c','d','e','f','g','h'], BUCKETS )
plt.show()    

## Testing the priority buffer for Prioritized Exp. Replay

In [55]:
import numpy as np
from navigation.dqn.utils import prioritybuffer

pbuffer = prioritybuffer.PriorityBuffer( 128, 0 )
pbuffer.add( [0.0, 1.0], 0, [0.1, 1.1], 0, False )

pbuffer._maxpriority

1

In [56]:
## see here, a warning due to non-zero division (it's taking min over all possible values in data buffer)
## have to fix by keeping the min somewhere else and updated, and then use it for this calculation
## one way: keep a variable for the running min (O(1) to grab, O(BatchSize) to update-> np.min->batch)
## another way: keep a mintree that will return the min value in O(log(n) + k)
_states, _actions, _nextStates, _rewards, _endflags, _indicesBatch, _isWeights = pbuffer.sample( 1 )

print( 'states: ', _states )
print( 'actions: ', _actions )
print( 'nextstates: ', _nextStates )
print( 'rewards: ', _rewards )
print( 'endflags: ', _endflags )
print( 'treeindices: ', _indicesBatch )
print( 'importance sampling weights: ', _isWeights )

states:  [[0. 1.]]
actions:  [0]
nextstates:  [[0.1 1.1]]
rewards:  [0]
endflags:  [0]
treeindices:  [127]
importance sampling weights:  [1.]


In [57]:
pbuffer.add( [0.2, 1.2], 1, [0.1, 1.1], 0, False )

In [58]:
_states, _actions, _nextStates, _rewards, _endflags, _indicesBatch, _isWeights = pbuffer.sample( 1 )

print( 'states: ', _states )
print( 'actions: ', _actions )
print( 'nextstates: ', _nextStates )
print( 'rewards: ', _rewards )
print( 'endflags: ', _endflags )
print( 'treeindices: ', _indicesBatch )
print( 'importance sampling weights: ', _isWeights )

states:  [[0. 1.]]
actions:  [0]
nextstates:  [[0.1 1.1]]
rewards:  [0]
endflags:  [0]
treeindices:  [127]
importance sampling weights:  [1.]


In [59]:
pbuffer.updatePriorities( np.array( [128] ), np.array( [100.] ) )

In [60]:
_states, _actions, _nextStates, _rewards, _endflags, _indicesBatch, _isWeights = pbuffer.sample( 1 )

print( 'states: ', _states )
print( 'actions: ', _actions )
print( 'nextstates: ', _nextStates )
print( 'rewards: ', _rewards )
print( 'endflags: ', _endflags )
print( 'treeindices: ', _indicesBatch )
print( 'importance sampling weights: ', _isWeights )

states:  [[0.2 1.2]]
actions:  [1]
nextstates:  [[0.1 1.1]]
rewards:  [0]
endflags:  [0]
treeindices:  [128]
importance sampling weights:  [0.33850375]


In [61]:
pbuffer._maxpriority

15.84988284150878

## Testing segmentrees from various implementations

* Adaptation from openai-baselines + other sources
* OpenAI-baselines

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from navigation.dqn.utils import sumtree
from navigation.dqn.utils import segmentree

In [ ]:
smtree1 = sumtree.SumTree(8)
smtree2 = segmentree.SumTree(8)

_nodevals = { 'a' : 3, 'b' : 10, 'c' : 12, 'd' : 4, 'e' : 1, 'f' : 2, 'g' : 8, 'h' : 2 }

for key in _nodevals.keys() :
    smtree1.add( key, _nodevals[key] )
    smtree2.add( key, _nodevals[key] )


In [ ]:
smtree1.total()

In [ ]:
smtree2.sum()

### checking update functionality

In [ ]:
# data, tree and sum before the update
print( 'data: ', smtree2._data  )
print( 'tree: ', smtree2._tree )
print( 'sum: ', smtree2.sum() )

# do an update (change 'a' from nodeval 3 to nodeval 5)
smtree2.update( 7, 5 )

# data, tree and sum after the update
print( 'data: ', smtree2._data  )
print( 'tree: ', smtree2._tree )
print( 'sum: ', smtree2.sum() )

### checking the probability distribution generated from both

In [ ]:
NSAMPLES = 100000
BUCKETS1 = [0] * len( smtree1._data )
BUCKETS2 = [0] * len( smtree2._data )
for _ in range( NSAMPLES ) :
    _rnd = np.random.random()
    
    _indx, _, _data = smtree1.getNode( smtree1.total() * _rnd )
    BUCKETS1[_indx - len( smtree1._data ) + 1] += 1
    
    _indx, _, _data = smtree2.getNode( smtree2.sum() * _rnd )
    BUCKETS2[_indx - len( smtree2._data ) + 1] += 1
    
fig1, ax1 = plt.subplots()
ax1.bar( ['a','b','c','d','e','f','g','h'], BUCKETS1 )

fig2, ax2 = plt.subplots()
ax2.bar( ['a','b','c','d','e','f','g','h'], BUCKETS2 )
plt.show()    

### testing the mintree

In [ ]:
mintree = segmentree.MinTree(8)
mintree.add( 'a', 1 )
mintree.add( 'b', 3 )
mintree.add( 'c', 5 )
mintree.add( 'd', 0 )

print( 'min: ', mintree.min() )

mintree.add( 'e', -1 )

print( 'min: ', mintree.min() )

mintree.add( 'f', 10 )

print( 'min: ', mintree.min() )

### testing openai-baselines segmentree

In [ ]:
from navigation.dqn.utils import openai_segmentree as op_segmentree

In [ ]:
# a simple test case
smtree3 = op_segmentree.SumSegmentTree(8)

smtree3[0] = 3
smtree3[1] = 10
smtree3[2] = 12
smtree3[3] = 4
smtree3[4] = 1
smtree3[5] = 2
smtree3[6] = 8
smtree3[7] = 2

print( 'sum: ', smtree3.sum() )

In [ ]:
_sum1 = smtree1.total()
_sum2 = smtree2.sum()
_sum3 = smtree3.sum()

assert (_sum1 == _sum2) and (_sum2 == _sum3), 'ERROR> sumtrees should return same total cumsum'

_nodeval = _sum1 * np.random.random()

_indx1, _, _ = smtree1.getNode( _nodeval )
_indx2, _, _ = smtree2.getNode( _nodeval )
_indx3 = smtree3.find_prefixsum_idx( _nodeval ) + smtree3._capacity - 1

print( 'indx1: ', _indx1 )
print( 'indx2: ', _indx2 )
print( 'indx3: ', _indx3 )